### Setup

This notebook performs the following setup actions for this example use of Amazon SageMaker Feature Store:

1. Create online-only feature groups
2. Create source and destination Amazon Managed Streaming for Kafka (MSK) topics.
3. Create an Amazon Kinesis Data Applications (KDA) application that aggregates data from source topic and loads to the target topic.

**Recommended settings to run this notebook in SageMaker Studio:**

- Image: Data Science
- Kernel: Python3
- Instance type: <font color='blue'>ml.m5.large (2 vCPU + 8 GiB)</font>

**Important Note:**

DO NOT "Run All Cells" on this notebook, manual steps are needed for successful execution.

### Get ARN's of Lambda functions from CloudFormation stack outputs
1. InvokeFraudEndpointLambdaARN
2. StreamingAggLambdaARN

Be sure to set the stack name in cell below to match the name of the CloudFormation stack used to create this SageMaker domain. If you used the default stack name, you should not need to make any updates

![SegmentLocal](images/get-cloudformation-stack-name.gif "get_cf_name")

In [22]:
STACK_NAME = 'sagemaker-featurestore-msk-kda-stack' # if you're not using the default stack name, replace this
%store STACK_NAME

Stored 'STACK_NAME' (str)


In [23]:
import sys
import boto3

cf_client = boto3.client('cloudformation')

try:
    outputs = cf_client.describe_stacks(StackName=STACK_NAME)['Stacks'][0]['Outputs']
    for o in outputs:
        if o['OutputKey'] == 'IngestLambdaFunctionARN':
            # lambda_to_fs_arn = o['OutputValue']
            lambda_to_fs_arn = o['OutputValue']
        if o['OutputKey'] == 'PredictLambdaFunctionARN':
            # lambda_to_model_arn = o['OutputValue']
            lambda_to_model_arn = o['OutputValue']
        if o['OutputKey'] == 'PredictLambdaFunctionName':
            # predict_lambda_name = o['OutputValue']
            predict_lambda_name = o['OutputValue']
        if o['OutputKey'] == 'AggregateFeatureStoreNameOutput':
            aggregate_feature_group_name = o['OutputValue']        
        if o['OutputKey'] == 'AggregateBatchFeatureStoreNameOutput':
            aggregate_batch_feature_group_name = o['OutputValue']
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [3]:
print(f'lambda_to_model_arn: {lambda_to_model_arn}')
print(f'lambda_to_fs_arn: {lambda_to_fs_arn}')
print(f'predict_lambda_name: {predict_lambda_name}')
print(f'aggregate_feature_group_name: {aggregate_feature_group_name}')
print(f'aggregate_batch_feature_group_name: {aggregate_batch_feature_group_name}')

lambda_to_model_arn: arn:aws:lambda:ap-southeast-1:850995562355:function:InvokeFraudEndpointLambda
lambda_to_fs_arn: arn:aws:lambda:ap-southeast-1:850995562355:function:StreamingIngestAggFeatures
predict_lambda_name: InvokeFraudEndpointLambda
aggregate_feature_group_name: cc-agg-fg
aggregate_batch_feature_group_name: cc-agg-batch-fg


In [4]:
try:
    response = cf_client.describe_stack_resource(
        StackName=STACK_NAME,
        LogicalResourceId='MSKCluster'
    )
    MSKClusterArn = response["StackResourceDetail"]["PhysicalResourceId"]
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [5]:
print(f'msk_cluster_arn: {MSKClusterArn}')

msk_cluster_arn: arn:aws:kafka:ap-southeast-1:850995562355:cluster/sagemaker-featurestore-msk-kda-stack-msk-cluster/d0b50f00-568a-4797-9895-b232f2a0d24b-3


In [6]:
%store lambda_to_model_arn

Stored 'lambda_to_model_arn' (str)


In [7]:
%store predict_lambda_name

Stored 'predict_lambda_name' (str)


In [8]:
%store aggregate_feature_group_name

Stored 'aggregate_feature_group_name' (str)


In [9]:
%store aggregate_batch_feature_group_name

Stored 'aggregate_batch_feature_group_name' (str)


In [10]:
%store MSKClusterArn

Stored 'MSKClusterArn' (str)


In [11]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

### Imports and other setup

In [12]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json

role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker')
smfs_runtime = boto3.Session().client(service_name='sagemaker-featurestore-runtime')

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Create online-only feature groups
When using Amazon SageMaker Feature Store, a core design decision is the definition of feature groups. For our credit card fraud detection use case, we have decided to use two of them:

1. aggregate_feature_group_name - holds aggregate features that will be updated in near real-time (streaming ingestion)
2. aggregate_batch_feature_group_name - holds aggregate features that will be updated in batch

Establishing a feature group is a one-time step and is done using the `CreateFeatureGroup` API. 

Feature groups can be created as **online-only**, **offline-only**, or both **online and offline**, which replicates updates from an online store to an offline store in Amazon S3. Since our focus in this example is on demonstrating the use of the feature store for online inference and streaming aggregation of features, we make each of our feature groups online-only.

In addition to a feature group name, we provide metadata about each feature in the group. We are using a json file to define the schema, but this is not a requirement. We use a schema file to demonstrate how you might capture the feature group definitions, enabling you to recreate them consistently as you move from a development environment to a test or production environment. In our schema file, we also highlight the record identifier and the event timestamp. All feature groups must have these two features, but you get to decide how to name them.

Here is a visual summary of the feature groups we will create below.

![SegmentLocal](images/feature_groups.png "connection")

#### cc-agg-fg schema

In [13]:
!pygmentize schema/clicks-agg-fg-schema.json

{
  "description": "Aggregated real-time features for each user, updated via streaming ingestion",
  "features": [
    {
      "name": "user_id",
      "type": "bigint",
      "description": "Unique identifier for the user"
    },
    {
      "name": "clicks_last_5m",
      "type": "bigint",
      "description": "Number of clicks on products in the last 5 minutes"
    },
    {
      "name": "page_views_last_5m",
      "type": "bigint",
      "description": "Number of page views in the last 5 minutes"
    },
    {
      "name": "orders_last_5m",
      "type": "bigint",
      "description": "Number of orders placed in the last 5 minutes"
    },
    {
      "name": "event_time",
      "type": "double",
      "description": "Timestamp of the metric event (epoch time in seconds)"
    }
  ],
  "record_identifier_feature_name": "user_id",
  "event_time_feature_name": "event_time",
  "tags": [
    {"Key": "Environment", "Value": "DEV"},
    {"Key": "IngestionType", "Value": "Streaming"},
    {

#### cc-agg-batch-fg schema

In [14]:
!pygmentize schema/clicks-agg-batch-fg-schema.json

{
  "description": "Aggregated batch features for each user, updated via batch processing",
  "features": [
    {
      "name": "user_id",
      "type": "bigint",
      "description": "Unique identifier for the user"
    },
    {
      "name": "total_orders_last_1w",
      "type": "bigint",
      "description": "Total number of orders placed by the user in the last 1 week"
    },
    {
      "name": "avg_order_value_last_1w",
      "type": "double",
      "description": "Average order value for the user over the last 1 week"
    },
    {
      "name": "total_interaction_value_last_1w",
      "type": "double",
      "description": "Aggregate interaction score (e.g., weighted sum of clicks, views, etc.) over the last 1 week"
    },
    {
      "name": "event_time",
      "type": "double",
      "description": "Timestamp of the batch aggregation event (epoch time in seconds)"
    }
  ],
  "record_identifier_feature_name": "user_id",
  "event_time_feature_name": "event_time",
  "tags": [
 

#### Utility functions to simplify creation of feature groups
`schema_to_defs` takes our schema file and returns feature definitions, and the names of the record identifier and event timestamp feature.

In [15]:
def schema_to_defs(filename):
    schema = json.loads(open(filename).read())
    
    feature_definitions = []
    
    for col in schema['Features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_definitions.append(feature)

    return feature_definitions, schema['record_identifier_feature_name'], schema['event_time_feature_name']

`schema_to_fg` creates a feature group from a schema file. If no s3 URI is passed, an online-only feature group is created.

In [16]:
def create_feature_group_from_schema(filename, fg_name, role_arn=None, s3_uri=None):
    schema = json.loads(open(filename).read())
    
    feature_defs = []
    
    for col in schema['features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_defs.append(feature)

    record_identifier_name = schema['record_identifier_feature_name']
    event_time_name = schema['event_time_feature_name']

    if role_arn is None:
        role_arn = get_execution_role()

    if s3_uri is None:
        offline_config = {}
    else:
        offline_config = {'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': s3_uri}}}
        
    sm.create_feature_group(
    # sm.update_feature_group(
        FeatureGroupName = fg_name,
        RecordIdentifierFeatureName = record_identifier_name,
        EventTimeFeatureName = event_time_name,
        FeatureDefinitions = feature_defs,
        Description = schema['description'],
        Tags = schema['tags'],
        OnlineStoreConfig = {'EnableOnlineStore': True},
        RoleArn = role_arn,
        **offline_config)

#### Create the two feature groups

In [17]:
create_feature_group_from_schema('schema/clicks-agg-fg-schema.json', aggregate_feature_group_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 create_feature_group_from_schema('schema/clicks-agg-fg-schema.json', aggregate_feature_g     │
│   2                                                                                              │
│                                                                                                  │
│ in create_feature_group_from_schema:27                                                           │
│                                                                                                  │
│   24 │   else:                                                                                   │
│   25 │   │   offline_config = {'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': s3_uri}}}     │
│   26 │                                                                                           │
│ ❱ 27 │   sm.create_feature_group(                                                                │
│   28 │   # sm.update_feature_group(                                                              │
│   29 │   │   FeatureGroupName = fg_name,                                                         │
│   30 │   │   RecordIdentifierFeatureName = record_identifier_name,                               │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ResourceInUse: An error occurred (ResourceInUse) when calling the CreateFeatureGroup operation: Resource Already 
Exists: FeatureGroup with name cc-agg-fg already exists. Choose a different name.

In [ ]:
create_feature_group_from_schema('schema/clicks-agg-batch-fg-schema.json', aggregate_batch_feature_group_name)

#### Show that the feature store is aware of the new feature groups

In [ ]:
sm.list_feature_groups()

#### Describe each feature group
Note that each feature group gets its own ARN, allowing you to manage IAM policies that control access to individual feature groups. The feature names and types are displayed, and the record identifier and event time features are called out specifically. Notice that there is only an `OnlineStoreConfig` and no `OfflineStoreConfig`, as we have decided not to replicate features offline for these groups.

In [18]:
sm.describe_feature_group(FeatureGroupName=aggregate_feature_group_name)

{'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-1:850995562355:feature-group/cc-agg-fg',
 'FeatureGroupName': 'cc-agg-fg',
 'RecordIdentifierFeatureName': 'user_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'user_id', 'FeatureType': 'Integral'},
  {'FeatureName': 'clicks_last_5m', 'FeatureType': 'Integral'},
  {'FeatureName': 'page_views_last_5m', 'FeatureType': 'Integral'},
  {'FeatureName': 'orders_last_5m', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 3, 8, 15, 32, 47, 977000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'ThroughputConfig': {'ThroughputMode': 'OnDemand'},
 'RoleArn': 'arn:aws:iam::850995562355:role/sagemaker-featurestore-msk-k-SageMakerExecutionRole-KkzfCCfZTIut',
 'FeatureGroupStatus': 'Created',
 'Description': 'Aggregated real-time features for each user, updated via streaming ingestion',
 'OnlineStoreTotalSize

In [19]:
sm.describe_feature_group(FeatureGroupName=aggregate_batch_feature_group_name)

{'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-1:850995562355:feature-group/cc-agg-batch-fg',
 'FeatureGroupName': 'cc-agg-batch-fg',
 'RecordIdentifierFeatureName': 'user_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'user_id', 'FeatureType': 'Integral'},
  {'FeatureName': 'total_orders_last_1w', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_order_value_last_1w', 'FeatureType': 'Fractional'},
  {'FeatureName': 'total_interaction_value_last_1w',
   'FeatureType': 'Fractional'},
  {'FeatureName': 'event_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 3, 8, 15, 32, 49, 308000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'ThroughputConfig': {'ThroughputMode': 'OnDemand'},
 'RoleArn': 'arn:aws:iam::850995562355:role/sagemaker-featurestore-msk-k-SageMakerExecutionRole-KkzfCCfZTIut',
 'FeatureGroupStatus': 'Created',
 'Description': 'Aggregated batch features for each user, updated via b

## Create Source and Destination Kafka topics

##### Why install Apache Kafka Libraries?

Amazon Managed Streaming for Apache Kafka (MSK) lets you use Apache Kafka data-plane operations to create topics and to produce and consume data. In MSK, You can use the AWS Management Console, the AWS Command Line Interface (AWS CLI), or the APIs in the SDK to perform control-plane operations. But since Kafka topic creation is a data-plane activity, we install Apache Kafka client libraries from a separate SageMaker Studio terminal window and execute it. For more details on how MSK works, refer to https://docs.aws.amazon.com/msk/latest/developerguide/what-is-msk.html. The required network and VPC configuration is already setup via the CloudFormation template.

Execute below commands in a separate terminal window in this SageMaker domain. Detailed instructions on how to create topics are available at https://docs.aws.amazon.com/msk/latest/developerguide/create-topic.html.

##### Retrieve MSK cluster connection string

The MSK cluster required would be created via the CloudFormation template. Retrieve the connection string for the pre-created MSK cluster using steps shown in this animation. Save off the connection string in a temporary location.

![SegmentLocal](images/get-msk-cluster-connection-string.gif "connection")

##### Apache Kafka client installation from SageMaker Studio terminal window


1. Navigate to SageMaker Studio environment.
2. In the top menu, hit "File" and choose "New" -> "Terminal".
3. Once a terminal window is fully available, execute commands in below cells. 
4. Replace the *cluster-connection-string* in commands below with Kafka connection string previously saved off in a temporary location.

##### Prepare the terminal environment

```
sudo yum -y update
sudo yum -y install java-1.8.0
sudo yum -y install wget
```

##### Create directory for Apache Kafka client download

```
mkdir kafka
chmod 777 kafka
cd kafka
```

##### Download and install Apache Kafka client

```
sudo wget https://archive.apache.org/dist/kafka/2.6.2/kafka_2.12-2.6.2.tgz
tar -xzf kafka_2.12-2.6.2.tgz
cd kafka_2.12-2.6.2
```

##### Create source topic (replace cluster-connection-string with MSK cluster connection string stored in temporary location)
```
bin/kafka-topics.sh --create --bootstrap-server <cluster-connection-string> --replication-factor 2 --partitions 1 --topic cctopic
```

##### Create destination topic (replace cluster-connection-string with connection string stored in temporary location)

```
bin/kafka-topics.sh --create --bootstrap-server <cluster-connection-string> --replication-factor 2 --partitions 1 --topic ccdesttopic
```

##### Verify created topics (replace cluster-connection-string with connection string stored in temporary location)

```
bin/kafka-topics.sh --bootstrap-server <cluster-connection-string> --list
```

##### To make cleanup easier remove the kafka client. Re-install using steps above if it had to be referred again.

```
cd ..
rm -rf kafka_2.12-2.6.2
rm -rf kafka_2.12-2.6.2.tgz
```

##### Exit out of the terminal window

```
exit
```

## Map the Kafka source topic as an event source for Lambda fraud detection

In [20]:
lambda_client = boto3.client('lambda')

source_topic_name = 'cctopic2'

lambda_client.create_event_source_mapping(EventSourceArn=MSKClusterArn,
                                          FunctionName=lambda_to_model_arn,
                                          StartingPosition='LATEST',
                                          Enabled=True,
                                          Topics=[source_topic_name]
                                         ) #DestinationConfig would handle discarded record

{'ResponseMetadata': {'RequestId': '7a416fed-4600-41ad-9c0d-9874fd3ea3bc',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 09 Mar 2025 04:53:17 GMT',
   'content-type': 'application/json',
   'content-length': '1296',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7a416fed-4600-41ad-9c0d-9874fd3ea3bc'},
  'RetryAttempts': 0},
 'UUID': '59d135c2-6ba4-473a-acda-9ac6d5b6bf9c',
 'StartingPosition': 'LATEST',
 'BatchSize': 100,
 'EventSourceArn': 'arn:aws:kafka:ap-southeast-1:850995562355:cluster/sagemaker-featurestore-msk-kda-stack-msk-cluster/d0b50f00-568a-4797-9895-b232f2a0d24b-3',
 'FunctionArn': 'arn:aws:lambda:ap-southeast-1:850995562355:function:InvokeFraudEndpointLambda',
 'LastModified': datetime.datetime(2025, 3, 9, 4, 53, 17, 462000, tzinfo=tzlocal()),
 'LastProcessingResult': 'No records processed',
 'State': 'Creating',
 'StateTransitionReason': 'USER_INITIATED',
 'Topics': ['cctopic'],
 'AmazonManagedKafkaEventSourceConfig': {'ConsumerGroupId': '59d135c2-6ba4-

## Map the Kafka destination topic as an event source for Lambda streaming ingest features

In [21]:
lambda_client = boto3.client('lambda')

dest_topic_name = 'ccdesttopic2'

lambda_client.create_event_source_mapping(EventSourceArn=MSKClusterArn,
                                          FunctionName=lambda_to_fs_arn,
                                          StartingPosition='LATEST',
                                          Enabled=True,
                                          Topics=[dest_topic_name]
                                         ) #DestinationConfig would handle discarded records

{'ResponseMetadata': {'RequestId': 'd0027ceb-6d66-4b8b-8443-cdee7cda6b47',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 09 Mar 2025 04:53:19 GMT',
   'content-type': 'application/json',
   'content-length': '1301',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd0027ceb-6d66-4b8b-8443-cdee7cda6b47'},
  'RetryAttempts': 0},
 'UUID': 'e14ea835-2954-4027-90c1-7cd40fcb44ef',
 'StartingPosition': 'LATEST',
 'BatchSize': 100,
 'EventSourceArn': 'arn:aws:kafka:ap-southeast-1:850995562355:cluster/sagemaker-featurestore-msk-kda-stack-msk-cluster/d0b50f00-568a-4797-9895-b232f2a0d24b-3',
 'FunctionArn': 'arn:aws:lambda:ap-southeast-1:850995562355:function:StreamingIngestAggFeatures',
 'LastModified': datetime.datetime(2025, 3, 9, 4, 53, 19, 335000, tzinfo=tzlocal()),
 'LastProcessingResult': 'No records processed',
 'State': 'Creating',
 'StateTransitionReason': 'USER_INITIATED',
 'Topics': ['ccdesttopic'],
 'AmazonManagedKafkaEventSourceConfig': {'ConsumerGroupId': 'e14ea835-

## Create Amazon Kinesis Data Analytics for Apache Flink application

Now that we have source/destination MSK topics and have them hooked up to the respective Lambda functions, let us focus on creating a Amazon Kinesis Data Analytics for Apache Flink application using a pre-created Apache Zeppelin studio environment.

The MSK cluster connection string saved off in a temporary location would be needed again for steps below.

#### Open the pre-created KDA Studio Environment

For new projects, we recommend that you use the new Kinesis Data Analytics Studio. Kinesis Data Analytics Studio combines ease of use with advanced analytical capabilities, which makes it possible to build sophisticated stream processing applications in minutes. Let’s see how that works in practice. Studio notebooks for Kinesis Data Analytics allows you to interactively query data streams in real time, and easily build and run stream processing applications using standard SQL, Python, and Scala. With a few clicks in the AWS Management console, you can launch a serverless notebook to query data streams and get results in seconds. In steps below, we navigate to the Kinesis console, run the pre-created KDA Studio environment, upload sample notebook, key in MSK connection strings, build and deploy an executable KDA streaming application.

![SegmentLocal](images/start-open-kda-studio-environment.gif "studio")

##### Download pre-created Zeppelin Notebook

Download the pre-built Zeppelin notebook (from GitHub) into your local environment. This notebook defines the schema for source and destination Kafka topics. In addition, it also defines the logic for aggregating data in source topic and feeds into the destination topic using Apache Flink Streaming SQL. To learn more on how this works, refer to https://docs.aws.amazon.com/kinesisanalytics/latest/java/how-zeppelin-interactive.html.

![SegmentLocal](images/download-zeppelin-notebook.gif "zpln_download")

##### Upload pre-created Zeppelin Notebook

Upload the pre-built Zeppelin notebook (from GitHub) into this Studio environment. 

<font color='red'>Do not execute invdividual cells in the imported notebook. The notebook would be built and deployed directly from Zeppelin environment.</font> 

![SegmentLocal](images/upload-kda-msk-flink-note-zeppelin.gif "zpln_upload")

##### Add MSK cluster connection string to cells

Using Apache Kafka client libraries, the source and target topics were created in the MSK cluster previously from SageMaker Studio terminal. In this step, the MSK cluster's connection string (saved off in a temporary location) is keyed into the Zeppelin notebook cells. In these cells, the notebook creates the metadata for the topics which is registered via AWS Glue Schema Registry.

![SegmentLocal](images/key-in-msk-cluster-connection-string.gif "zpln_connection_string")

##### Build KDA app from Zeppelin Notebook

Building a Kinesis Data Analytics Application by building the Zeppelin notebook. This step approximately takes about 5 minutes.

![SegmentLocal](images/build-kda-app.gif "zpln_build_app")

##### Deploy KDA app and run

Now the application code is built and uploaded to S3. Next steps are to create a KDA application from it and start it. Starting a Kinesis Data Analytics application typically takes several minutes. Ensure that the CloudFormation created role that has text "KDAStreamingApplicationExecutionRole" completely or partly in its name.

![SegmentLocal](images/deploy-run-kda-app.gif "zpln_deploy_app")